In [1]:
from lxml import etree
import re
import os
import sqlite3

In [28]:
conn = sqlite3.connect('rechtspraak.db')
c = conn.cursor()

In [35]:
c2 = conn.cursor()
c2.execute(''' DROP TABLE uitspraken_links_ljn''')
c2.execute(''' CREATE TABLE uitspraken_links_ljn
            (id text, 
            reference text,
            ljn text,
            cnt integer
            )
        ''')

In [13]:
ljn_regex = '''LJN ([A-Z]{2}[0-9]{4})'''

In [14]:
re.findall(ljn_regex, 'LJN BF0918')

['BF0918']

In [23]:
import pandas as pd
ljn_to_ecli_file = '/media/sf_VBox_Shared/CaseLaw/WasWordt.csv'
ljn_to_ecli_df = pd.read_csv(ljn_to_ecli_file, header=None)
ljn_to_ecli_df.columns = ['ljn', 'ecli', 'type']
ljn_to_ecli_df = ljn_to_ecli_df[ljn_to_ecli_df['type']=='Uitspraak'] # Only take uitspraken
ljn_to_ecli_df.head()

,ljn,ecli,type
0,AA1000,ECLI:NL:RBARN:1999:AA1000,Uitspraak
1,AA1001,ECLI:NL:RBARN:1999:AA1001,Uitspraak
2,AA1002,ECLI:NL:RBARN:1999:AA1002,Uitspraak
3,AA1003,ECLI:NL:RBARN:1999:AA1003,Uitspraak
4,AA1004,ECLI:NL:RBARN:1999:AA1004,Uitspraak


In [24]:
ljn_to_ecli_dict = {row['ljn']: row['ecli'] for i, row in ljn_to_ecli_df.iterrows()}

In [25]:
len(ljn_to_ecli_dict.keys())

529870

In [36]:
from collections import Counter

rows = c.execute('SELECT id, text from uitspraken')

for row in rows:
    ecli, text = row
    matches = re.findall(ljn_regex, text)
    references = [(ljn_to_ecli_dict[ljn], ljn) for ljn in matches if ljn in ljn_to_ecli_dict]
    if len(references)>0:
        for (other, ljn), cnt in Counter(references).most_common():
            if ecli != other: # no self references
                query = ''' INSERT INTO uitspraken_links_ljn
                    VALUES (?, ?, ?, ?)
                    '''
                c2.execute(query, (ecli, other, ljn, cnt))

In [37]:
#How many links do we have?
c.execute('''select count(distinct id), count(*) from uitspraken_links_ljn''').fetchall()

[(2701, 4174)]

In [39]:
conn.commit()
conn.close()